<a href="https://colab.research.google.com/github/KanikaAdik/text_classification/blob/main/MailCheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 6.9 MB/s 
     |████████████████████████████████| 42 kB 1.7 MB/s 
     |████████████████████████████████| 660 kB 68.2 MB/s 
     |████████████████████████████████| 457 kB 68.5 MB/s 
     |████████████████████████████████| 10.1 MB 54.0 MB/s 
     |████████████████████████████████| 181 kB 72.9 MB/s 
     |████████████████████████████████| 58 kB 7.4 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.2.0
    Uninstalling typing-extensions-4.2.0:
      Successfully uninstalled typing-extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srs

In [4]:
import pandas as pd
import spacy
from spacy.util import minibatch
from spacy.training import Example
from google.colab import drive
from google.colab import drive

#drive.mount('/content/drive')

#drive.mount('/Colab_Notebooks/')
drive.mount('/csv/')
df = pd.read_csv("spam.csv")
df.head(10)

Drive already mounted at /csv/; to attempt to forcibly remount, call drive.mount("/csv/", force_remount=True).


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


# New Section

# New Section

In [5]:
nlp = spacy.blank("en") #creating a blank model
category = nlp.add_pipe("textcat") #adding textcat type of category
#add labels to classifier
category.add_label("ham")
category.add_label("spam")

1

In [6]:
#training model
train_texts = df['text'].values
train_labels = [{'cats':{'ham': label =="ham",
                         'spam': label =="spam"}} for label in df['label']]
train_data = list(zip(train_texts, train_labels))
train_data[:5]

[('Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
  {'cats': {'ham': True, 'spam': False}}),
 ('Ok lar... Joking wif u oni...', {'cats': {'ham': True, 'spam': False}}),
 ("Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
  {'cats': {'ham': False, 'spam': True}}),
 ('U dun say so early hor... U c already then say...',
  {'cats': {'ham': True, 'spam': False}}),
 ("Nah I don't think he goes to usf, he lives around here though",
  {'cats': {'ham': True, 'spam': False}})]

In [5]:
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=8) # created mini batches to train model
for batch in batches:
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer)

In [11]:
import random
random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

batches = minibatch(train_data, size=8) # created mini batches to train model in batches of 8
losses ={}
for epoch in range(10):
  random.shuffle(train_data)
  batches = minibatch(train_data, size=8)
  for batch in batches:
    for text, labels in batch:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, labels)
        nlp.update([example], sgd=optimizer, losses=losses)
  print(losses)


{'textcat': 146.49582210897245}
{'textcat': 208.39032440893416}
{'textcat': 253.63403016954126}
{'textcat': 277.0189795004773}
{'textcat': 290.16163219344185}
{'textcat': 298.5811517769604}
{'textcat': 312.2094990638064}
{'textcat': 318.85702952374595}
{'textcat': 327.2029261215402}
{'textcat': 331.2526222278101}


In [8]:
random.seed(1)
spacy.util.fix_random_seed(1)
optimizer = nlp.begin_training()

losses = {}
for epoch in range(10):
    random.shuffle(train_data)
    # Create the batch generator with batch size = 8
    batches = minibatch(train_data, size=8)
    # Iterate through minibatches
    for batch in batches:
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            nlp.update([example], sgd=optimizer, losses=losses)
    print(losses)

{'textcat': 136.21752654929605}
{'textcat': 198.21212421938657}
{'textcat': 234.79118735974748}
{'textcat': 260.6598102853204}
{'textcat': 278.32411491260206}
{'textcat': 285.39782853846776}
{'textcat': 296.06841954534144}
{'textcat': 307.04272443476214}
{'textcat': 312.21163010525873}
{'textcat': 316.25898668121704}


In [9]:
texts = ["Are you ready for the tea party????? It's gonna be wild",
         "URGENT Reply to this message for GUARANTEED FREE TEA" ]
docs = [nlp.tokenizer(text) for text in texts]
    
# Use textcat to get the scores for each doc
textcat = nlp.get_pipe('textcat')
scores = textcat.predict(docs)

print(scores)

[[9.9999988e-01 1.4828471e-07]
 [9.9999821e-01 1.8324829e-06]]


In [10]:
predicted_labels = scores.argmax(axis=1)
print([textcat.labels[label] for label in predicted_labels])

['ham', 'ham']


# New Section